In [1]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
from keras import layers, models, optimizers
from sklearn import model_selection, preprocessing
from sklearn.metrics import accuracy_score
#import sklearn.metrics.accuracy_score 
from sklearn.metrics import accuracy_score

/home/kriti/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = open('corpus').read()
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    labels.append(content[0])
    texts.append(content[1:])

# create a dataframe using texts and lables
df = pd.DataFrame()
df['text'] = texts
df['label'] = labels

In [3]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['text'], df['label'])
## encode the label using lael encoder
encoder = preprocessing.LabelEncoder()
#df['label'] = encoder.fit_transform(df['label'].fillna('0'))
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [7]:
embeddings_index = {}
for i, line in enumerate(open('wiki-news-300d-1M.vec')):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')
            
            
            

In [8]:
from keras.preprocessing import text, sequence
token = text.Tokenizer()
token.fit_on_texts(df['text'])
word_index = token.word_index

In [9]:
word_index

{'the': 1,
 'and': 2,
 'a': 3,
 'i': 4,
 'to': 5,
 'of': 6,
 'this': 7,
 'is': 8,
 'it': 9,
 'in': 10,
 'for': 11,
 'that': 12,
 'was': 13,
 'not': 14,
 'you': 15,
 'but': 16,
 'with': 17,
 'on': 18,
 'my': 19,
 'have': 20,
 'book': 21,
 'as': 22,
 'are': 23,
 'be': 24,
 'one': 25,
 'so': 26,
 'if': 27,
 'very': 28,
 'like': 29,
 'all': 30,
 'at': 31,
 'just': 32,
 'they': 33,
 'from': 34,
 'about': 35,
 'would': 36,
 'an': 37,
 'great': 38,
 'good': 39,
 'or': 40,
 'read': 41,
 'has': 42,
 'what': 43,
 'by': 44,
 'had': 45,
 'when': 46,
 'get': 47,
 'will': 48,
 'more': 49,
 'out': 50,
 "it's": 51,
 'only': 52,
 'movie': 53,
 'your': 54,
 'me': 55,
 'no': 56,
 'can': 57,
 'his': 58,
 'there': 59,
 'some': 60,
 'really': 61,
 "don't": 62,
 'who': 63,
 'he': 64,
 'up': 65,
 'than': 66,
 'it.': 67,
 'even': 68,
 'because': 69,
 'her': 70,
 'first': 71,
 'do': 72,
 'much': 73,
 'other': 74,
 'time': 75,
 'were': 76,
 'after': 77,
 'we': 78,
 'she': 79,
 'how': 80,
 '-': 81,
 'been': 82,
 

In [10]:
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=70)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen = 70)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


In [11]:
embedding_matrix.shape

(68898, 300)

In [12]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return accuracy_score(predictions, valid_y)

In [13]:
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_cnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print("CNN, Word Embeddings",  accuracy)

Epoch 1/1
7500/7500 [==============================] - 16s 2ms/step - loss: 0.6056
CNN, Word Embeddings 0.5072
